# cohttp_lwt_example

A simple example of use of the DuckDuckGo API server by `cohttp.lwt`.

In [1]:
#require "core" ;;
#require "cohttp.lwt" ;;
#require "cohttp.top" ;;
#require "yojson" ;;

/home/opam/.opam/4.04.1/lib/base/caml: added to search path
/home/opam/.opam/4.04.1/lib/base/caml/caml.cma: loaded
/home/opam/.opam/4.04.1/lib/base/shadow_stdlib: added to search path
/home/opam/.opam/4.04.1/lib/base/shadow_stdlib/shadow_stdlib.cma: loaded
/home/opam/.opam/4.04.1/lib/sexplib/0: added to search path
/home/opam/.opam/4.04.1/lib/sexplib/0/sexplib0.cma: loaded
/home/opam/.opam/4.04.1/lib/base: added to search path
/home/opam/.opam/4.04.1/lib/base/base.cma: loaded
/home/opam/.opam/4.04.1/lib/ocaml/unix.cma: loaded
/home/opam/.opam/4.04.1/lib/ocaml/bigarray.cma: loaded
/home/opam/.opam/4.04.1/lib/fieldslib: added to search path
/home/opam/.opam/4.04.1/lib/fieldslib/fieldslib.cma: loaded
/home/opam/.opam/4.04.1/lib/ppx_compare/runtime-lib: added to search path
/home/opam/.opam/4.04.1/lib/ppx_compare/runtime-lib/ppx_compare_lib.cma: loaded
/home/opam/.opam/4.04.1/lib/sexplib: added to search path
/home/opam/.opam/4.04.1/lib/sexplib/sexplib.cma: loaded
/home/opam/.opam/4.04.1/l

In [2]:
open Core ;;
open Lwt.Infix ;;
open Cohttp ;;
open Cohttp_lwt_unix ;;

The default asynchronous context implemented in `cohttp.lwt` cannot work on IOCaml.
We define a custom context based on blocking API.

In [3]:
let ctx =
  let open Caml.Unix in
  let service name =
    let s = getservbyname name "tcp" in
    Lwt.return (Some Resolver.({ name; port = s.s_port; tls = (name = "https" || name = "imaps"); })) in
  let resolver service uri =
    let host = match Uri.host uri with Some h -> h | None -> "localhost" in
    let port = match Uri.port uri with Some p -> p | None -> service.Resolver.port in
    match getaddrinfo host (string_of_int port) [AI_SOCKTYPE SOCK_STREAM] with
    | [] -> Lwt.return (`Unknown ("name resolution failed"))
    | {ai_addr=ADDR_INET (addr,port);_}::_ -> Lwt.return (`TCP (Ipaddr_unix.of_inet_addr addr, port))
    | {ai_addr=ADDR_UNIX file;_}::_ -> Lwt.return (`Unix_domain_socket file) in
  let resolver = Resolver_lwt.init ~service ~rewrites:["", resolver] () in
  Cohttp_lwt_unix_net.({ default_ctx with resolver })

val ctx : Cohttp_lwt_unix_net.ctx =
 {Cohttp_lwt_unix_net.ctx = <abstr>; resolver = <abstr>}

We obtain the meaning of a given query from the DuckDuckGo API server.

[Yojson](https://github.com/mjambon/yojson) is a JSON serialization/deserialization library written in OCaml, and
[Yojson.Basic.Util](https://mjambon.github.io/mjambon2016/yojson-doc/Yojson.Basic.Util.html) contains utilities for parsing a JSON.

In [4]:
let search_query query =
  let base_uri = Uri.of_string "http://api.duckduckgo.com/?format=json" in
  let uri = Uri.add_query_param base_uri ("q", [query]) in
  Client.get ~ctx uri >>= fun (resp, body) -> (* GET contents from a given uri *)
  assert (Response.status resp = `OK) ; (* Check HTTP response code *)
  Cohttp_lwt_body.to_string body >|= fun body -> (* Receive contents *)
  let json = Yojson.Basic.from_string body in (* Parse a JSON *)
  (* Find field "Definition" or "Abstract" in a JSON *)
  let def = Yojson.Basic.Util.(json |> member "Definition" |> to_string_option) in
  let abs = Yojson.Basic.Util.(json |> member "Abstract" |> to_string_option) in
  match def with
  | None | Some "" -> abs
  | _ -> def

val search_query : string -> string option Lwt.t = <fun>

In [5]:
Lwt_main.run (search_query "ocaml")

- : string option =
Some
 "OCaml, originally named Objective Caml, is the main implementation of the programming language Caml, created by Xavier Leroy, J\195\169r\195\180me Vouillon, Damien Doligez, Didier R\195\169my, Asc\195\161nder Su\195\161rez and others in 1996. A member of the ML language family, OCaml extends the core Caml language with object-oriented programming constructs."